<a href="https://colab.research.google.com/github/SunkaraboinaPraveenKumar/Machine_Learning_Projects/blob/main/Tools_with_LLM_Concurrency_Conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [104]:
!pip install -q langchain-groq langchain-core requests

In [105]:
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests
import os

In [106]:
from google.colab import  userdata
os.environ['GROQ_API_KEY']=userdata.get('groq_api_key')

In [107]:
# tool create
@tool
def multiply(a:int,b:int)->int:
  """Given 2 numbers returns their product"""
  return a*b

In [108]:
print(multiply.invoke({'a':3,'b':2}))

6


In [109]:
multiply.args

{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

In [110]:
# tool bind
llm = ChatGroq(
    model="llama-3.3-70b-versatile"
)

In [111]:
llm_with_tools = llm.bind_tools([multiply])

In [112]:
llm_with_tools.invoke("Hi")

AIMessage(content="I'm here to help. Is there something I can help you with, or would you like to use one of the available functions?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 221, 'total_tokens': 249, 'completion_time': 0.11232657, 'prompt_time': 0.014478059, 'queue_time': 0.219029264, 'total_time': 0.126804629}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_3f3b593e33', 'finish_reason': 'stop', 'logprobs': None}, id='run-e092d413-3b78-4932-8f39-ad7d8f997637-0', usage_metadata={'input_tokens': 221, 'output_tokens': 28, 'total_tokens': 249})

In [113]:
llm_with_tools.invoke("Multiply 3 and 2").tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 2},
  'id': 'call_36ys',
  'type': 'tool_call'}]

In [114]:
llm_with_tools.invoke("Multiply 3 and 2").tool_calls[0]

{'name': 'multiply',
 'args': {'a': 3, 'b': 2},
 'id': 'call_qj6c',
 'type': 'tool_call'}

In [115]:
# tool execution
tool_call = llm_with_tools.invoke("Multiply 3 and 2").tool_calls[0]

In [116]:
print(tool_call)

{'name': 'multiply', 'args': {'a': 3, 'b': 2}, 'id': 'call_h4yd', 'type': 'tool_call'}


In [117]:
multiply.invoke(tool_call['args'])

6

In [118]:
multiply.invoke(tool_call)

ToolMessage(content='6', name='multiply', tool_call_id='call_h4yd')

In [119]:
query = HumanMessage("Can you multiply 3 and 2?")
messages = [query]

In [120]:
messages

[HumanMessage(content='Can you multiply 3 and 2?', additional_kwargs={}, response_metadata={})]

In [121]:
result = llm_with_tools.invoke(messages)

In [122]:
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_xzm1', 'function': {'arguments': '{"a": 3, "b": 2}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 229, 'total_tokens': 248, 'completion_time': 0.069090909, 'prompt_time': 0.017088341, 'queue_time': 0.218533645, 'total_time': 0.08617925}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_3f3b593e33', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-4e28ea98-df08-4e8d-b9e6-064a0a59f0de-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 2}, 'id': 'call_xzm1', 'type': 'tool_call'}], usage_metadata={'input_tokens': 229, 'output_tokens': 19, 'total_tokens': 248})

In [123]:
messages.append(result)

In [124]:
messages

[HumanMessage(content='Can you multiply 3 and 2?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_xzm1', 'function': {'arguments': '{"a": 3, "b": 2}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 229, 'total_tokens': 248, 'completion_time': 0.069090909, 'prompt_time': 0.017088341, 'queue_time': 0.218533645, 'total_time': 0.08617925}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_3f3b593e33', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-4e28ea98-df08-4e8d-b9e6-064a0a59f0de-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 2}, 'id': 'call_xzm1', 'type': 'tool_call'}], usage_metadata={'input_tokens': 229, 'output_tokens': 19, 'total_tokens': 248})]

In [125]:
tool_result = multiply.invoke(result.tool_calls[0])

In [126]:
messages.append(tool_result)

In [127]:
messages

[HumanMessage(content='Can you multiply 3 and 2?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_xzm1', 'function': {'arguments': '{"a": 3, "b": 2}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 229, 'total_tokens': 248, 'completion_time': 0.069090909, 'prompt_time': 0.017088341, 'queue_time': 0.218533645, 'total_time': 0.08617925}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_3f3b593e33', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-4e28ea98-df08-4e8d-b9e6-064a0a59f0de-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 2}, 'id': 'call_xzm1', 'type': 'tool_call'}], usage_metadata={'input_tokens': 229, 'output_tokens': 19, 'total_tokens': 248}),
 ToolMessage(content='6', name='multiply', tool_call_id='call_xzm1')]

In [128]:
llm_with_tools.invoke(messages)

AIMessage(content='The product of 3 and 2 is 6.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 259, 'total_tokens': 272, 'completion_time': 0.047272727, 'prompt_time': 0.024293279, 'queue_time': 0.217618893, 'total_time': 0.071566006}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_6507bcfb6f', 'finish_reason': 'stop', 'logprobs': None}, id='run-d1f66f67-479f-49b3-a62e-d84f427a170d-0', usage_metadata={'input_tokens': 259, 'output_tokens': 13, 'total_tokens': 272})

In [129]:
# Currency Conversion Tool

In [130]:
from langchain_core.tools import InjectedToolArg
from typing import Annotated
@tool
def get_conversion_factor(base_currency:str,target_currency:str)->float:
  """Given a base currency and target currency returns the conversion factor"""
  url = f'https://v6.exchangerate-api.com/v6/c754eab14ffab33112e380ca/pair/{base_currency}/{target_currency}'

  response = requests.get(url)

  return response.json()

@tool
def convert(base_currency_value:int,conversion_rate:Annotated[float, InjectedToolArg])->float:
  """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """
  return base_currency_value*conversion_rate

In [131]:
convert.args

{'base_currency_value': {'title': 'Base Currency Value', 'type': 'integer'},
 'conversion_rate': {'title': 'Conversion Rate', 'type': 'number'}}

In [132]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1745798401,
 'time_last_update_utc': 'Mon, 28 Apr 2025 00:00:01 +0000',
 'time_next_update_unix': 1745884801,
 'time_next_update_utc': 'Tue, 29 Apr 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 85.4741}

In [133]:
convert.invoke({'base_currency_value':10, 'conversion_rate':85.16})

851.5999999999999

In [134]:
llm = ChatGroq(
    model = "llama-3.3-70b-versatile"
)

In [135]:
llm_with_tools = llm.bind_tools([get_conversion_factor,convert])

In [136]:
messages = [HumanMessage("What is the conversion factor between INR and USD?")]

In [137]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD?', additional_kwargs={}, response_metadata={})]

In [138]:
ai_message = llm_with_tools.invoke(messages)

In [139]:
messages.append(ai_message)

In [140]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'INR', 'target_currency': 'USD'},
  'id': 'call_ejfb',
  'type': 'tool_call'}]

In [141]:
import json

for tool_call in ai_message.tool_calls:
  if tool_call['name']=='get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)

    conversion_rate = json.loads(tool_message1.content)['conversion_rate']

    messages.append(tool_message1)

  if tool_call['name']=='convert':
   tool_call['args']['conversion_rate'] = conversion_rate
   tool_message2 = convert.invoke(tool_call)
   messages.append(tool_message2)

In [142]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ejfb', 'function': {'arguments': '{"base_currency": "INR", "target_currency": "USD"}', 'name': 'get_conversion_factor'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 324, 'total_tokens': 348, 'completion_time': 0.087272727, 'prompt_time': 0.022537881, 'queue_time': 0.221071522, 'total_time': 0.109810608}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_9a8b91ba77', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d8e35c13-6a9d-415f-a479-c243afe9625d-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'base_currency': 'INR', 'target_currency': 'USD'}, 'id': 'call_ejfb', 'type': 'tool_call'}], usage_metadata={'input_tokens': 324, 'output_tokens': 24, 'total_tokens': 348}),
 ToolMessage(content='{"result": "succe

In [143]:
llm_with_tools.invoke(messages).content

'The conversion factor between INR and USD is 0.0117. This means that 1 INR is equivalent to 0.0117 USD.'

In [144]:
from langchain.agents import initialize_agent, AgentType

# Step 5: Initialize the Agent ---
agent_executor = initialize_agent(
    tools=[get_conversion_factor, convert],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  # using ReAct pattern
    verbose=True  # shows internal thinking
)

In [145]:
# --- Step 6: Run the Agent ---
user_query = "Hi how are you?"

response = agent_executor.invoke({"input": user_query})



> Entering new AgentExecutor chain...
Thought: The user is greeting me, and I should respond with a friendly message.
Action:
```
{
  "action": "Final Answer",
  "action_input": "I'm doing well, thank you for asking! How can I assist you today?"
}
```


> Finished chain.


In [146]:
user_query = "What is the conversion factor between INR and USD?"

response = agent_executor.invoke({"input": user_query})



> Entering new AgentExecutor chain...
Thought: To find the conversion factor, we need to use the get_conversion_factor tool. This tool takes two parameters: base_currency and target_currency.

Action:
```
{
  "action": "get_conversion_factor",
  "action_input": {
    "base_currency": "INR",
    "target_currency": "USD"
  }
}
```

Observation: {'result': 'success', 'documentation': 'https://www.exchangerate-api.com/docs', 'terms_of_use': 'https://www.exchangerate-api.com/terms', 'time_last_update_unix': 1745798401, 'time_last_update_utc': 'Mon, 28 Apr 2025 00:00:01 +0000', 'time_next_update_unix': 1745884801, 'time_next_update_utc': 'Tue, 29 Apr 2025 00:00:01 +0000', 'base_code': 'INR', 'target_code': 'USD', 'conversion_rate': 0.0117}
Thought:The conversion factor between INR and USD has been retrieved successfully. The conversion rate is 0.0117, which means 1 INR is equivalent to 0.0117 USD.

Action:
```
{
  "action": "Final Answer",
  "action_input": "The conversion factor between I

In [147]:
user_query = "What is the 1000INR in USD?"

response = agent_executor.invoke({"input": user_query})



> Entering new AgentExecutor chain...
Thought: To convert 1000 INR to USD, we need to know the conversion factor between INR and USD. 

Action:
```
{
  "action": "get_conversion_factor",
  "action_input": {
    "base_currency": "INR",
    "target_currency": "USD"
  }
}
```

Observation: {'result': 'success', 'documentation': 'https://www.exchangerate-api.com/docs', 'terms_of_use': 'https://www.exchangerate-api.com/terms', 'time_last_update_unix': 1745798401, 'time_last_update_utc': 'Mon, 28 Apr 2025 00:00:01 +0000', 'time_next_update_unix': 1745884801, 'time_next_update_utc': 'Tue, 29 Apr 2025 00:00:01 +0000', 'base_code': 'INR', 'target_code': 'USD', 'conversion_rate': 0.0117}
Thought:Now that we have the conversion factor, we can use it to convert 1000 INR to USD.

Action:
```
{
  "action": "convert",
  "action_input": {
    "base_currency_value": 1000,
    "conversion_rate": 0.0117
  }
}
```

Observation: 11.700000000000001
Thought:I know what to respond

Action:
```
{
  "action":